In [19]:
import pandas as pd
import numpy as np
#show all columns
pd.set_option('display.max_columns', None)

german_datasets = pd.DataFrame(columns=["text", "label"])
german_datasets = german_datasets.astype({"text": str, "label": np.float32})

In [20]:
news1 = pd.read_csv("data/german/news/RP-Mod.csv")
news1
german_datasets = pd.concat([german_datasets, news1], ignore_index=True)

In [21]:
news2 = pd.read_csv("data/german/news/RP-Crowd-1.csv")
news2
german_datasets = pd.concat([german_datasets, news1[["text", "label"]]], ignore_index=True)

In [22]:
refugee = pd.read_csv("data/german/refugee/german hatespeech refugees.csv")
refugee["text"] = refugee["Tweet"]
refugee["label"] = (refugee["Hatespeech Rating (Expert 2)"]-1)/5
refugee
german_datasets = pd.concat([german_datasets, refugee[["text", "label"]]], ignore_index=True)

In [23]:
comments_df = pd.read_csv("data/german/foreigners/comments.csv")
annotated_comments_df = pd.read_csv("data/german/foreigners/annotated_comments.csv")

# Group by 'comment_id' and calculate the mean valence for each 'comment_id'
grouped_df = annotated_comments_df.groupby("comment_id")["valence"].mean().reset_index()

# Merge the dataframes on 'comment_id'
final_df = pd.merge(comments_df, grouped_df, on="comment_id", how="inner")

# Rename columns to 'text' and 'label'
final_df = final_df.rename(columns={"message": "text", "valence": "label"})

# keep only the columns 'text' and 'label'
final_df = final_df[["text", "label"]]

final_df["label"] = final_df["label"]-1

german_datasets = pd.concat([german_datasets, final_df], ignore_index=True)

In [24]:
hasoc = pd.read_csv("data/german/hasoc/german_dataset.tsv", sep="\t")
# hasoc["task_1"] is always either NOT or HOF
hasoc["label"] = hasoc["task_1"].map({"NOT": 0, "HOF": 1})

german_datasets = pd.concat([german_datasets, hasoc[["text", "label"]]], ignore_index=True)

In [25]:
germeval2018 = pd.read_csv("data/german/germeval2018/germeval2018.training.txt", sep="\t", header=None)
germeval2018.columns = ["text", "label", "label2"]
germeval2018["label"] = germeval2018["label"].map({"OTHER": 0, "OFFENSE": 1})
germeval2018["origin"] = "germeval"

german_datasets = pd.concat([german_datasets, germeval2018[["text", "label", "origin"]]], ignore_index=True)

germeval2018 = pd.read_csv("data/german/germeval2018/germeval2018.test.txt", sep="\t", header=None)
germeval2018.columns = ["text", "label", "label2"]
germeval2018["label"] = germeval2018["label"].map({"OTHER": 0, "OFFENSE": 1})
germeval2018["origin"] = "germeval"

german_datasets = pd.concat([german_datasets, germeval2018[["text", "label", "origin"]]], ignore_index=True)

In [26]:
germeval2019 = pd.read_csv("data/german/germeval2019/Shared-Task-2019_Data_germeval2019.training_subtask1_2.txt", sep="\t", header=None)
germeval2019.columns = ["text", "label", "label2"]
germeval2019["label"] = germeval2019["label"].map({"OTHER": 0, "OFFENSE": 1})
germeval2019["origin"] = "germeval"

german_datasets = pd.concat([german_datasets, germeval2019[["text", "label", "origin"]]], ignore_index=True)

germeval2019 = pd.read_csv("data/german/germeval2019/fz.h-da.de_fileadmin_user_upload_germeval2019GoldLabelsSubtask1_2.txt", sep="\t", header=None)
germeval2019.columns = ["text", "label", "label2"]
germeval2019["label"] = germeval2019["label"].map({"OTHER": 0, "OFFENSE": 1})
germeval2019["origin"] = "germeval"

german_datasets = pd.concat([german_datasets, germeval2019[["text", "label", "origin"]]], ignore_index=True)

In [27]:
german_datasets

,text,label,origin
0,das alles ist wie Selbstbefriedigung...denn sc...,1.0,NaN
1,Na Gott sei Dank!!! Wen soll er denn auch besc...,1.0,NaN
2,"Naja, er kann sich ja schlecht daneben stellen...",1.0,NaN
3,Oh Gott - jetzt lässt Kellermann auch noch den...,1.0,NaN
4,Mal gespannt. als Ausrede warum er niemals be...,1.0,NaN
...,...,...,...
57346,Es fand aber nie eine Emanzipierungs-Phase der...,0.0,germeval
57347,Um es klar zu stellen: Ich will hier kein Whit...,0.0,germeval
57348,Und dann habe ich da noch die McArthur-Briefe ...,0.0,germeval
57349,al sehen wer der Ersatzmann wird. Hier könnte ...,0.0,germeval


In [28]:
# remove duplicated text
german_datasets = german_datasets.drop_duplicates(subset=["text"])
german_datasets = german_datasets.reset_index(drop=True)

#remove nan text
german_datasets = german_datasets.dropna(subset=["text"])
german_datasets

,text,label,origin
0,das alles ist wie Selbstbefriedigung...denn sc...,1.0,NaN
1,Na Gott sei Dank!!! Wen soll er denn auch besc...,1.0,NaN
2,"Naja, er kann sich ja schlecht daneben stellen...",1.0,NaN
3,Oh Gott - jetzt lässt Kellermann auch noch den...,1.0,NaN
4,Mal gespannt. als Ausrede warum er niemals be...,1.0,NaN
...,...,...,...
34554,Es fand aber nie eine Emanzipierungs-Phase der...,0.0,germeval
34555,Um es klar zu stellen: Ich will hier kein Whit...,0.0,germeval
34556,Und dann habe ich da noch die McArthur-Briefe ...,0.0,germeval
34557,al sehen wer der Ersatzmann wird. Hier könnte ...,0.0,germeval


In [29]:
import re
from bs4 import BeautifulSoup
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    # print(text)
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

german_datasets["text"] = german_datasets["text"].apply(text_cleaning)

c:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [30]:
#shuffle the data
german_datasets = german_datasets.sample(frac=1).reset_index(drop=True)
german_datasets.to_parquet("german_datasets.parquet")

In [31]:
germeval = german_datasets[german_datasets["origin"] == "germeval"]
pretrain = german_datasets[german_datasets["origin"] != "germeval"]



In [32]:
germeval

,text,label,origin
0,@Gaumen_freude Danke schön und schönes Foto,0.0,germeval
5,@de_sputnik @Saudi Arabien habt ihr mal wieder...,1.0,germeval
6,@tagesschau Wo bleibt die Hetze gegen Marcon??,0.0,germeval
7,@lkasachsen @OtschiHorst Lächerlich die Aufreg...,0.0,germeval
8,Hier! Ey! |LBR| VERPISST EUCH! |LBR| der alte ...,1.0,germeval
...,...,...,...
34546,Ich bin ja eigentlich noch gar nicht bereit .....,0.0,germeval
34549,@SirSchnee @rstrohmi Für deutsche Kriminalität...,1.0,germeval
34550,"@ardmoma Wer sich in Gefahr begibt, kommt dari...",0.0,germeval
34553,@PolizeiSachsen @kruger_elke Das reicht nie! D...,1.0,germeval


In [33]:
pretrain

,text,label,origin
1,"und warum stehen hier mal wieder alle ( 6x) ""M...",1.0,NaN
2,Was hat dieser kommerzielle Merchandising-Müll...,1.0,NaN
3,"Ja, wenn man manche Argumente liest, fragt man...",0.0,NaN
4,Ich fordere grundsätzlich -nicht zuletzt auch-...,0.0,NaN
9,..... Der Beitrag würde nicht den AGB entsprec...,1.0,NaN
...,...,...,...
34551,Wieder das Lieblingsthema unserer ach so armen...,1.0,NaN
34552,"Könnte es vielleicht auch daran liegen, dass s...",1.0,NaN
34554,"Da es sich um Vorhaben von Klöckner handelt, k...",0.0,NaN
34556,Wer kann mir sagen wenn ich eine Partei wähle ...,0.0,NaN


In [34]:
print(germeval["label"].value_counts())
print(pretrain["label"].value_counts())

0.0    10334
1.0     5091
Name: label, dtype: int64
0.000000    10932
1.000000     7837
0.200000      107
0.500000       80
0.600000       75
0.400000       68
0.800000       30
0.750000        2
0.333333        1
0.666667        1
Name: label, dtype: int64


In [35]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(germeval, test_size=0.5)

In [36]:
#save the files to parquet
train.to_parquet("train.parquet")
test.to_parquet("test.parquet")
pretrain.to_parquet("pretrain.parquet")

In [37]:
for (id, row) in train.iterrows():
    print(row["text"])
    print(row["label"])
    print("----------")

@hortarimar @Georg_Pazderski Sich für muslimische Feiertage einzusetzen, bedeutet nicht gleich religiös zu sein. Es bedeutet sich für die im Grundgesetz verankerte Religionsfreiheit einzusetzen. Alles andere über unsere Organisation finden Sie unter: tgd.de, dann kommt es auch zu weniger Missverständnissen.
0.0
----------
#blockupy @Blockupy Polizei setzt wieder massiv Pfefferspray und Knüppel ein
0.0
----------
@clyde261282 @DenkerWerner @Beatrix_vStorch @netzpolitik_org Du bist lustig. Kann man dich auch buchen?
0.0
----------
Wo Liebe ist, da tritt als Begleiterscheinung die Freude auf, so Kard. #Marx in der Predigt zur Amtsübergabe des Diözesanjugendpfarrers
0.0
----------
@ImTmee @panda_munich O_o |LBR| Was hast du bitte als Suchfilter eingegeben..???
0.0
----------
@gaborhalasz1 Ihre Frage ist ironisch - richtig?
0.0
----------
@ollimoe Mich besorgt das Säbelrasseln gegen RU. Ich kenne das russ. Militär v. 1974-1988 sehr gut u. glauben Sie, d. NATO ist KEIN Gegner.
0.0
----------

0.0
----------
Guten Morgen an alle, die hier schon länger leben |LBR| Eure Kanzlerin
0.0
----------
@Martin_Lejeune wann bringen die dich in die Klappse... du Nullnummer!?
1.0
----------
@rspctfl @ergroovt @Innenwelttramp @ThomasMichael71 @houelle_beck @ksemann2 @DrKassandraPari Gut. Dann sind halt Männer benachteiligt. Macht es auch nicht besser.
0.0
----------
@Gooney75 ... um Familien geht es nicht. Keiner Partei.
0.0
----------
@Ralf_Stegner Ralle du alter Mülleimer lebst ja immer noch...dachte das zeitliche hat dich gesegnet... |LBR| Bist halt eine zähe Unkrautwurzel
1.0
----------
@Opa_Antifa Alles alles Gute und viel Kraft!
0.0
----------
@sigmargabriel @EmmanuelMacron Auf des deutschen Steuerzahlers Kosten!
0.0
----------
@ArmesDeutschla6 @Hartmut73 @jouwatch War doch vorraus zu sehen was die Linksfaschisten machen kotze gleich
1.0
----------
@FranzHuber_in @gegrilltebanane @kokaxcola ja witziger trollaccount haha lol halts maul jetzt
1.0
----------
Berühmteste Putzfrau Deutsc